In [1]:
import os
import urllib.request
from concurrent.futures import ThreadPoolExecutor
from google.cloud import storage
import time

In [2]:
BUCKET_NAME = "nkzoomcamp_hw3_2025"  

In [3]:
CREDENTIALS_FILE = "keys.json"  
client = storage.Client.from_service_account_json(CREDENTIALS_FILE)

In [4]:
BASE_URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-"
MONTHS = [f"{i:02d}" for i in range(1, 7)] 
DOWNLOAD_DIR = "."

In [5]:
CHUNK_SIZE = 8 * 1024 * 1024  

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

bucket = client.bucket(BUCKET_NAME)

In [6]:
def download_file(month):
    url = f"{BASE_URL}{month}.parquet"
    file_path = os.path.join(DOWNLOAD_DIR, f"yellow_tripdata_2024-{month}.parquet")

    try:
        print(f"Downloading {url}...")
        urllib.request.urlretrieve(url, file_path)
        print(f"Downloaded: {file_path}")
        return file_path
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        return None


def verify_gcs_upload(blob_name):
    return storage.Blob(bucket=bucket, name=blob_name).exists(client)


def upload_to_gcs(file_path, max_retries=3):
    blob_name = os.path.basename(file_path)
    blob = bucket.blob(blob_name)
    blob.chunk_size = CHUNK_SIZE  
    
    for attempt in range(max_retries):
        try:
            print(f"Uploading {file_path} to {BUCKET_NAME} (Attempt {attempt + 1})...")
            blob.upload_from_filename(file_path)
            print(f"Uploaded: gs://{BUCKET_NAME}/{blob_name}")
            
            if verify_gcs_upload(blob_name):
                print(f"Verification successful for {blob_name}")
                return
            else:
                print(f"Verification failed for {blob_name}, retrying...")
        except Exception as e:
            print(f"Failed to upload {file_path} to GCS: {e}")
        
        time.sleep(5)  
    
    print(f"Giving up on {file_path} after {max_retries} attempts.")


if __name__ == "__main__":
    with ThreadPoolExecutor(max_workers=4) as executor:
        file_paths = list(executor.map(download_file, MONTHS))

    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(upload_to_gcs, filter(None, file_paths))  # Remove None values

    print("All files processed and verified.")

Downloaded: ./yellow_tripdata_2024-02.parquet
Downloaded: ./yellow_tripdata_2024-01.parquet
Downloaded: ./yellow_tripdata_2024-04.parquet
Downloaded: ./yellow_tripdata_2024-03.parquet
Downloaded: ./yellow_tripdata_2024-06.parquet
Downloaded: ./yellow_tripdata_2024-05.parquet
Uploading ./yellow_tripdata_2024-01.parquet to nkzoomcamp_hw3_2025 (Attempt 1)...
Uploading ./yellow_tripdata_2024-02.parquet to nkzoomcamp_hw3_2025 (Attempt 1)...
Uploading ./yellow_tripdata_2024-03.parquet to nkzoomcamp_hw3_2025 (Attempt 1)...
Uploading ./yellow_tripdata_2024-04.parquet to nkzoomcamp_hw3_2025 (Attempt 1)...
Uploaded: gs://nkzoomcamp_hw3_2025/yellow_tripdata_2024-01.parquet
Verification successful for yellow_tripdata_2024-01.parquet
Uploading ./yellow_tripdata_2024-05.parquet to nkzoomcamp_hw3_2025 (Attempt 1)...
Uploaded: gs://nkzoomcamp_hw3_2025/yellow_tripdata_2024-02.parquet
Verification successful for yellow_tripdata_2024-02.parquet
Uploading ./yellow_tripdata_2024-06.parquet to nkzoomcamp_hw